# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
# Loading Packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
from datetime import datetime

In [2]:
# Checking current working directory
cwd = os.getcwd()
cwd

'D:\\Study\\ds\\Guided_capstone\\DS_big_mountain_ski_resort_capstone\\4-5-preprocessing-modeling'

In [3]:
# Specify the data folder path
data_path = '../0-data/'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [4]:
# Loading data into dataframe and inspecting first few lines
df = pd.read_csv(data_path+'step3_output.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,1,1,Eaglecrest Ski Area,Alaska,2600,1540,0,0.0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.00000,90.0,0.0,0
1,2,2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.00000,152.0,30.0,1
2,4,4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.00000,104.0,80.0,2
3,5,5,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,...,88.0,0.0,110.0,84.0,300.0,47.0,47.00000,107.0,0.0,2
4,8,8,Boreal Mountain Resort,California,7700,500,0,0.0,0,1,...,380.0,200.0,150.0,54.0,400.0,49.0,64.16681,150.0,200.0,2


In [5]:
# Dropping Duplicate column
df = df.drop('Unnamed: 0.1', axis=1)

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [6]:
df.dtypes

Unnamed: 0             int64
Name                  object
state                 object
summit_elev            int64
vertical_drop          int64
trams                  int64
fastEight            float64
fastSixes              int64
fastQuads              int64
quad                   int64
triple                 int64
double                 int64
surface                int64
total_chairs           int64
Runs                 float64
TerrainParks         float64
LongestRun_mi        float64
SkiableTerrain_ac    float64
Snow Making_ac       float64
daysOpenLastYear     float64
yearsOpen            float64
averageSnowfall      float64
AdultWeekday         float64
AdultWeekend         float64
projectedDaysOpen    float64
NightSkiing_ac       float64
clusters               int64
dtype: object

In [7]:
dfo = df.select_dtypes(include=['object']) # Select all object typr columns

In [8]:
df = pd.concat([df.drop('state', axis=1), pd.get_dummies(dfo['state'])], axis=1) # axis is to select column 

In [9]:
df.head()

,Unnamed: 0,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,1,Eaglecrest Ski Area,2600,1540,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,4,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0
3,5,Yosemite Ski & Snowboard Area,7800,600,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,8,Boreal Mountain Resort,7700,500,0,0.0,0,1,1,3,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.shape

(157, 60)

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [10]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X)

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [11]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [12]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [13]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = lm.predict(X_test)

In [14]:
y_pred

array([ 5.88364258e+01,  7.12114258e+01,  5.13559570e+01,  4.45356445e+01,
        5.44477539e+01, -4.87136849e+13,  5.33793945e+01,  5.02583008e+01,
        3.64184570e+01,  3.77241211e+01,  5.88569336e+01,  8.71879883e+01,
        4.57543945e+01,  6.40063477e+01,  6.91333008e+01,  7.56918945e+01,
       -4.87460464e+13,  8.24614258e+01,  6.17905273e+01,  6.11577148e+01,
        5.09887695e+01,  4.31538086e+01,  4.91225586e+01,  5.28969727e+01,
        5.66567383e+01,  8.79145508e+01,  4.84086914e+01,  4.49926758e+01,
       -4.87460465e+13,  4.60200195e+01,  4.43364258e+01,  5.12348633e+01,
        3.10981445e+01,  5.12895508e+01,  7.86997070e+01,  3.16850586e+01,
        4.83579102e+01,  7.80317383e+01,  5.98881836e+01,  6.09965820e+01])

## Review Model Outcomes — Iterate over additional models as needed

In [15]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
explained_variance_score(y_test, y_pred)

-6.429388173518345e+23

In [16]:
mean_absolute_error(y_test, y_pred)

3655144445256.3096

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [17]:
lm.intercept_

-931246992362.8306

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [18]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by='Coefficient',ascending=False).head(10)

,Coefficient
fastEight,5.487523e+13
fastSixes,4.482026e+13
trams,1.811590e+13
New York,1.432945e+13
California,1.343361e+13
Pennsylvania,1.295133e+13
Michigan,1.295133e+13
total_chairs,1.276543e+13
Wisconsin,1.244264e+13
New Hampshire,1.190415e+13


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

In [30]:
def RegressionModel(X,y, test_size, random_state):
    try:
        # Test Train split
        X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        # Fit the values into the model
        lm = linear_model.LinearRegression()
        model = lm.fit(X_train,y_train)
        # Predict model with test set
        y_pred = model.predict(X_test)
        
        explained_variance = explained_variance_score(y_test, y_pred)
        meanabsolute_error = mean_absolute_error(y_test, y_pred)
        intercept = lm.intercept_
        
        Coefficient = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by='Coefficient',ascending=False)
        
        results = {'Explained Variance' : explained_variance,
                    'Mean absolute error': meanabsolute_error,
                  'intercept': intercept,
                  'Co-efficient': Coefficient}
        return results
    except Exception as e:
        return str(e)

In [25]:
from sklearn.ensemble import RandomForestRegressor
def RandomforestregressorModel(X,y, random_state, estimators):
    # Test Train split
    X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
    # Define model
    regressor = RandomForestRegressor(random_state= random_state, n_estimators = estimators)
    model = regressor.fit(X_train, y_train)
    
    score = model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    
    explained_variance = explained_variance_score(y_test, y_pred)
    meanabsolute_error = mean_absolute_error(y_test, y_pred)
    
    Coefficient = pd.DataFrame(model.feature_importances_, X.columns, columns=['Coefficient']).sort_values(by='Coefficient',ascending=False)
    
    results = {
                'score' : score,
                'Explained Variance' : explained_variance,
                'Mean absolute error': meanabsolute_error,
                'Co-efficient': Coefficient
                }
    
    return results
    

In [26]:
RandomforestregressorModel(X,y, 0, 200)

{'score': 0.7393323964503332,
 'Explained Variance': 0.7396400219904755,
 'Mean absolute error': 6.351934363799286,
 'Co-efficient':                    Coefficient
 AdultWeekday          0.696852
 total_chairs          0.034608
 Snow Making_ac        0.029422
 vertical_drop         0.027949
 Unnamed: 0            0.025510
 yearsOpen             0.023486
 averageSnowfall       0.017374
 summit_elev           0.014884
 LongestRun_mi         0.013784
 Runs                  0.013459
 NightSkiing_ac        0.012069
 TerrainParks          0.011090
 daysOpenLastYear      0.010317
 projectedDaysOpen     0.010105
 SkiableTerrain_ac     0.009627
 clusters              0.006627
 triple                0.006521
 surface               0.005960
 quad                  0.005052
 New Hampshire         0.003395
 double                0.003284
 North Carolina        0.003134
 Michigan              0.003116
 Maine                 0.002270
 Vermont               0.001989
 Connecticut           0.001224
 fas

#### Model 2

In [27]:
df.columns

Index(['Unnamed: 0', 'Name', 'summit_elev', 'vertical_drop', 'trams',
       'fastEight', 'fastSixes', 'fastQuads', 'quad', 'triple', 'double',
       'surface', 'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'AdultWeekend', 'projectedDaysOpen',
       'NightSkiing_ac', 'clusters', 'Alaska', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Missouri', 'Montana', 'Nevada', 'New Hampshire', 'New Mexico',
       'New York', 'North Carolina', 'Ohio', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Dakota', 'Tennessee', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object')

In [28]:
X1 = df.drop(['Name','AdultWeekend','Alaska', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Missouri', 'Montana', 'Nevada', 'New Hampshire', 'New Mexico',
       'New York', 'North Carolina', 'Ohio', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Dakota', 'Tennessee', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'], axis=1)

scaler1 = preprocessing.StandardScaler().fit(X1)

X1_scaled=scaler1.transform(X1)

In [31]:
RegressionModel(X1,y, 0.25, 1)

{'Explained Variance': 0.7629771299226165,
 'Mean absolute error': 6.145809637866693,
 'intercept': 16.734478106092475,
 'Co-efficient':                     Coefficient
 clusters           4.894783e+00
 quad               1.367232e+00
 fastQuads          7.891293e-01
 TerrainParks       7.314663e-01
 AdultWeekday       6.655688e-01
 triple             6.624285e-01
 total_chairs       5.491366e-01
 double             4.200304e-01
 LongestRun_mi      3.012226e-01
 surface            2.713642e-01
 yearsOpen          4.604643e-02
 NightSkiing_ac     1.313457e-02
 averageSnowfall    8.151784e-03
 daysOpenLastYear   7.411079e-03
 projectedDaysOpen  5.600431e-03
 vertical_drop      4.239501e-03
 SkiableTerrain_ac  4.114939e-03
 Snow Making_ac     4.069336e-03
 Unnamed: 0         2.994124e-03
 Runs               3.125116e-04
 summit_elev        2.758969e-04
 trams              2.617698e-15
 fastEight          5.325601e-16
 fastSixes          3.885781e-16}

In [32]:
RandomforestregressorModel(X1,y, 0, 200)

{'score': 0.729017483444547,
 'Explained Variance': 0.7299987747251722,
 'Mean absolute error': 6.502580385304659,
 'Co-efficient':                    Coefficient
 AdultWeekday          0.698686
 total_chairs          0.035335
 Snow Making_ac        0.029235
 vertical_drop         0.028416
 yearsOpen             0.027825
 Unnamed: 0            0.027486
 averageSnowfall       0.018095
 summit_elev           0.015465
 LongestRun_mi         0.015125
 Runs                  0.014340
 NightSkiing_ac        0.014094
 TerrainParks          0.011739
 daysOpenLastYear      0.010741
 projectedDaysOpen     0.010218
 SkiableTerrain_ac     0.009974
 triple                0.007875
 clusters              0.007785
 surface               0.006601
 quad                  0.005416
 double                0.004663
 fastQuads             0.000887
 fastSixes             0.000000
 fastEight             0.000000
 trams                 0.000000}

**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [33]:
X2 = df.drop(['Name','AdultWeekend','Alaska', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Missouri', 'Montana', 'Nevada', 'New Hampshire', 'New Mexico',
       'New York', 'North Carolina', 'Ohio', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Dakota', 'Tennessee', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'summit_elev', 'vertical_drop'], axis=1)

scaler2 = preprocessing.StandardScaler().fit(X2)

X2_scaled=scaler2.transform(X2)

In [34]:
X2.head()

,Unnamed: 0,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,total_chairs,...,LongestRun_mi,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,projectedDaysOpen,NightSkiing_ac,clusters
0,1,0,0.0,0,0,0,0,4,0,4,...,2.0,640.0,60.0,45.0,44.0,350.0,47.0,90.0,0.0,0
1,2,0,0.0,0,0,0,1,0,2,3,...,1.0,30.0,30.0,150.0,36.0,69.0,30.0,152.0,30.0,1
2,4,0,0.0,0,1,2,3,1,0,7,...,1.2,800.0,80.0,115.0,49.0,250.0,74.0,104.0,80.0,2
3,5,0,0.0,0,0,0,1,3,1,5,...,0.4,88.0,0.0,110.0,84.0,300.0,47.0,107.0,0.0,2
4,8,0,0.0,0,1,1,3,1,2,8,...,1.0,380.0,200.0,150.0,54.0,400.0,49.0,150.0,200.0,2


In [35]:
RegressionModel(X2,y, 0.25, 1)

{'Explained Variance': 0.7692939420086848,
 'Mean absolute error': 6.164876418464734,
 'intercept': 15.885140104779971,
 'Co-efficient':                     Coefficient
 clusters           4.377702e+00
 quad               1.456155e+00
 LongestRun_mi      1.081759e+00
 TerrainParks       8.012111e-01
 AdultWeekday       7.082391e-01
 total_chairs       5.720826e-01
 surface            5.567869e-01
 triple             5.397564e-01
 double             4.688354e-01
 fastQuads          3.982066e-01
 yearsOpen          5.474066e-02
 Runs               4.931015e-02
 projectedDaysOpen  1.094858e-02
 NightSkiing_ac     9.433842e-03
 averageSnowfall    6.300170e-03
 Unnamed: 0         3.787333e-03
 SkiableTerrain_ac  9.593652e-04
 Snow Making_ac     1.437218e-04
 daysOpenLastYear   5.429443e-05
 trams              2.855875e-14
 fastSixes          7.438494e-15
 fastEight          2.914335e-15}

In [36]:
RandomforestregressorModel(X2,y, 0, 200)

{'score': 0.7204648565703902,
 'Explained Variance': 0.7215358103291184,
 'Mean absolute error': 6.691725949820793,
 'Co-efficient':                    Coefficient
 AdultWeekday          0.701671
 total_chairs          0.037185
 Snow Making_ac        0.034689
 Unnamed: 0            0.032191
 yearsOpen             0.031845
 LongestRun_mi         0.020432
 Runs                  0.018096
 averageSnowfall       0.016031
 NightSkiing_ac        0.014208
 SkiableTerrain_ac     0.013917
 projectedDaysOpen     0.012833
 TerrainParks          0.012622
 clusters              0.012321
 daysOpenLastYear      0.011242
 triple                0.009983
 surface               0.007507
 quad                  0.006902
 double                0.005305
 fastQuads             0.001021
 trams                 0.000000
 fastSixes             0.000000
 fastEight             0.000000}

## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | -6.429388173518345e+23 | 3655144445256.3096  |-|
| Model 2. | 0.7629771299226165 | 6.145809637866693 |'state'|
| Model 3. | 0.7692939420086848 | 6.164876418464734 |'state','summit_elev','vertical_drop'|

Model Selection: Both model 2 & Model 3 is nearly identical and giving almost same variance and MAE. Although both model looks same will take Model 3. because some features in the model 2 are not important to the problem statement. 